In [2]:

import duckdb; 
con=duckdb.connect('data/market/market.duckdb'); 
con.execute("SELECT * FROM ohlcv ORDER BY ts DESC LIMIT 1;").fetchall()


[('AAPL',
  'stock',
  'stooq_eod',
  datetime.datetime(2025, 8, 28, 0, 0),
  230.82,
  233.41,
  229.335,
  232.56,
  38074700.0,
  None)]

In [3]:
import pandas as pd
from pandas_datareader import data as pdr
df = pdr.DataReader("AAPL", "stooq", start="2010-01-01", end="2010-02-10").sort_index()
print(df.head(15))
print("\nRows:", len(df), " Unique days:", df.index.nunique())

               Open     High      Low    Close      Volume
Date                                                      
2010-01-04  6.42488  6.45501  6.39166  6.44034   583191777
2010-01-05  6.46388  6.48813  6.41768  6.45138   713715946
2010-01-06  6.45138  6.47712  6.34179  6.34840   652802003
2010-01-07  6.37037  6.38003  6.29093  6.33718   565387425
2010-01-08  6.33224  6.38003  6.29142  6.37983   530786106
2010-01-11  6.40911  6.40978  6.27291  6.32318   547235670
2010-01-12  6.29891  6.31243  6.21182  6.25133   704043769
2010-01-13  6.26294  6.34742  6.14214  6.33884   717292318
2010-01-14  6.32090  6.33371  6.29024  6.30208   513287899
2010-01-15  6.36190  6.36802  6.19486  6.19663   704839616
2010-01-19  6.27134  6.47583  6.23673  6.47157   865102093
2010-01-20  6.46767  6.48657  6.30455  6.37185   725332037
2010-01-21  6.38921  6.41895  6.23576  6.26117   720350400
2010-01-22  6.22305  6.24462  5.93242  5.95055  1040681451
2010-01-25  6.10410  6.16018  6.02447  6.11147  12339113

In [4]:
from datetime import datetime
def _fetch_stooq(symbol: str, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
    """Fetch daily bars from Stooq (ascending by date)."""
    df = pdr.DataReader(symbol, "stooq", start=start_dt.date(), end=end_dt.date())
    if df is None or df.empty:
        return pd.DataFrame()
    df = df.sort_index()  # ascending
    out = pd.DataFrame({
        "ts": pd.to_datetime(df.index).tz_localize(None),
        "open": pd.to_numeric(df.get("Open"), errors="coerce"),
        "high": pd.to_numeric(df.get("High"), errors="coerce"),
        "low": pd.to_numeric(df.get("Low"), errors="coerce"),
        "close": pd.to_numeric(df.get("Close"), errors="coerce"),
        "volume": pd.to_numeric(df.get("Volume"), errors="coerce"),
        "adj_close": pd.to_numeric(df.get("Adj Close"), errors="coerce") if "Adj Close" in df.columns else pd.NA,
    })
    out = out.dropna(subset=["ts","open","high","low","close"]).reset_index(drop=True)
    return out

In [8]:
from helpers import indicators

indicators.plot_rolling_returns(db_path="data/market/market.duckdb", symbol='NVDA', window_years=3)

/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:85: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'line': {'width': 2},
              'mode': 'lines',
              'name': '3-Year Rolling CAGR',
              'text': array(['2010-12-31 → 2013-12-31<br>CAGR: 2.27%',
                             '2011-12-31 → 2014-12-31<br>CAGR: 14.83%',
                             '2012-12-31 → 2015-12-31<br>CAGR: 41.64%',
                             '2013-12-31 → 2016-12-31<br>CAGR: 90.92%',
                             '2014-12-31 → 2017-12-31<br>CAGR: 115.04%',
                             '2015-12-31 → 2018-12-31<br>CAGR: 60.27%',
                             '2016-12-31 → 2019-12-31<br>CAGR: 30.61%',
                             '2017-12-31 → 2020-12-31<br>CAGR: 39.61%',
                             '2018-12-31 → 2021-12-31<br>CAGR: 107.0%',
                             '2019-12-31 → 2022-12-31<br>CAGR: 35.59%',
                             '2020-12-31 → 2023-12-31<br>CAGR: 56.08%',
                             '2021-12-31 → 2024-12-31<br>CAGR: 65.99%'], dtype=object),
              'type': 'scatter',
              'x': array(['2013-12-31T00:00:00.000000000', '2014-12-31T00:00:00.000000000',
                          '2015-12-31T00:00:00.000000000', '2016-12-31T00:00:00.000000000',
                          '2017-12-31T00:00:00.000000000', '2018-12-31T00:00:00.000000000',
                          '2019-12-31T00:00:00.000000000', '2020-12-31T00:00:00.000000000',
                          '2021-12-31T00:00:00.000000000', '2022-12-31T00:00:00.000000000',
                          '2023-12-31T00:00:00.000000000', '2024-12-31T00:00:00.000000000'],
                         dtype='datetime64[ns]'),
              'y': {'bdata': ('KVyPwvUoAkApXI/C9agtQFK4HoXr0U' ... 'G4HoXLQUAK16NwPQpMQI/C9Shcf1BA'),
                    'dtype': 'f8'}}],
    'layout': {'hovermode': 'x unified',
               'template': '...',
               'title': {'text': 'NVDA - 3-Year Rolling CAGR (Yearly)'},
               'xaxis': {'title': {'text': 'End Date'}},
               'yaxis': {'title': {'text': 'Rolling CAGR (%)'}}}
})

In [8]:
from helpers import indicators
fig, explain = indicators.plot_total_return_vs_benchmark(db_path="data/market/market.duckdb", symbol='MSFT', start_date="2020-01-01")

In [9]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'lines',
              'name': 'MSFT',
              'type': 'scatter',
              'x': array(['2020-01-02T00:00:00.000000', '2020-01-03T00:00:00.000000',
                          '2020-01-06T00:00:00.000000', ..., '2025-08-26T00:00:00.000000',
                          '2025-08-27T00:00:00.000000', '2025-08-28T00:00:00.000000'],
                         dtype='datetime64[us]'),
              'y': {'bdata': ('AAAAAACIw0Cy861UlErDQHgKk7hWV8' ... 'cdTojq30Bkqpb9ghvgQJ8OXSUcM+BA'),
                    'dtype': 'f8'}},
             {'mode': 'lines',
              'name': 'SPY',
              'type': 'scatter',
              'x': array(['2020-01-02T00:00:00.000000', '2020-01-03T00:00:00.000000',
                          '2020-01-06T00:00:00.000000', ..., '2025-08-26T00:00:00.000000',
                          '2025-08-27T00:00:00.000000', '2025-08-28T00:00:00.000000'],
                         dtype='datetime64[us]'),
              'y': {'bdata': ('AAAAAACIw0A/W3FWEWLDQFPwcgredM' ... 'u2lHb71EA7lXy9swfVQJe/7aXEGtVA'),
                    'dtype': 'f8'}}],
    'layout': {'hovermode': 'x unified',
               'template': '...',
               'title': {'text': 'Total Return: MSFT vs SPY (Growth of $10,000)'},
               'xaxis': {'title': {'text': 'Date'}},
               'yaxis': {'title': {'text': 'Value of $10,000 Investment ($)'}}}
})

In [10]:
explain

'If you had invested $10,000 in MSFT starting from 2020-01-01, it would have grown to **$33,176.88**, a total return of **231.77%**.\nIn comparison, the benchmark SPY would have grown to **$21,611.07**, a total return of **116.11%**.'

In [1]:
from helpers import indicators
indicators.get_best_worst_month_and_hit_rate(db_path="data/market/market.duckdb", symbol='AAPL')

/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:241: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = df['close'].resample('M').ffill().pct_change().dropna()
/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:242: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_returns = df['close'].resample('Y').ffill().pct_change().dropna()


{'best_month': '2020-08',
 'best_month_return': np.float64(21.65),
 'worst_month': '2018-11',
 'worst_month_return': np.float64(-18.12),
 'hit_rate': np.float64(59.36),
 'best_year': 2019,
 'best_year_return': np.float64(88.94),
 'worst_year': 2022,
 'worst_year_return': np.float64(-26.4)}

In [4]:
indicators.calculate_annualized_volatility("data/market/market.duckdb", "NVDA")
# Output: 22.45 (%)


/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:296: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df['close'].resample('M').last()


np.float64(45.16)

In [1]:
from helpers import indicators
fig, ex = indicators.plot_max_drawdown(db_path="data/market/market.duckdb", symbol="AAPL")

/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:362: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample("M").last().dropna()


In [2]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'fill': 'tozeroy',
              'mode': 'lines',
              'name': 'Drawdown (%)',
              'type': 'scatter',
              'x': array(['2010-01-31T00:00:00.000000', '2010-02-28T00:00:00.000000',
                          '2010-03-31T00:00:00.000000', '2010-04-30T00:00:00.000000',
                          '2010-05-31T00:00:00.000000', '2010-06-30T00:00:00.000000',
                          '2010-07-31T00:00:00.000000', '2010-08-31T00:00:00.000000',
                          '2010-09-30T00:00:00.000000', '2010-10-31T00:00:00.000000',
                          '2010-11-30T00:00:00.000000', '2010-12-31T00:00:00.000000',
                          '2011-01-31T00:00:00.000000', '2011-02-28T00:00:00.000000',
                          '2011-03-31T00:00:00.000000', '2011-04-30T00:00:00.000000',
                          '2011-05-31T00:00:00.000000', '2011-06-30T00:00:00.000000',
                          '2011-07-31T00:00:00.000000', '2011-08-31T00:00:00.000000',
                          '2011-09-30T00:00:00.000000', '2011-10-31T00:00:00.000000',
                          '2011-11-30T00:00:00.000000', '2011-12-31T00:00:00.000000',
                          '2012-01-31T00:00:00.000000', '2012-02-29T00:00:00.000000',
                          '2012-03-31T00:00:00.000000', '2012-04-30T00:00:00.000000',
                          '2012-05-31T00:00:00.000000', '2012-06-30T00:00:00.000000',
                          '2012-07-31T00:00:00.000000', '2012-08-31T00:00:00.000000',
                          '2012-09-30T00:00:00.000000', '2012-10-31T00:00:00.000000',
                          '2012-11-30T00:00:00.000000', '2012-12-31T00:00:00.000000',
                          '2013-01-31T00:00:00.000000', '2013-02-28T00:00:00.000000',
                          '2013-03-31T00:00:00.000000', '2013-04-30T00:00:00.000000',
                          '2013-05-31T00:00:00.000000', '2013-06-30T00:00:00.000000',
                          '2013-07-31T00:00:00.000000', '2013-08-31T00:00:00.000000',
                          '2013-09-30T00:00:00.000000', '2013-10-31T00:00:00.000000',
                          '2013-11-30T00:00:00.000000', '2013-12-31T00:00:00.000000',
                          '2014-01-31T00:00:00.000000', '2014-02-28T00:00:00.000000',
                          '2014-03-31T00:00:00.000000', '2014-04-30T00:00:00.000000',
                          '2014-05-31T00:00:00.000000', '2014-06-30T00:00:00.000000',
                          '2014-07-31T00:00:00.000000', '2014-08-31T00:00:00.000000',
                          '2014-09-30T00:00:00.000000', '2014-10-31T00:00:00.000000',
                          '2014-11-30T00:00:00.000000', '2014-12-31T00:00:00.000000',
                          '2015-01-31T00:00:00.000000', '2015-02-28T00:00:00.000000',
                          '2015-03-31T00:00:00.000000', '2015-04-30T00:00:00.000000',
                          '2015-05-31T00:00:00.000000', '2015-06-30T00:00:00.000000',
                          '2015-07-31T00:00:00.000000', '2015-08-31T00:00:00.000000',
                          '2015-09-30T00:00:00.000000', '2015-10-31T00:00:00.000000',
                          '2015-11-30T00:00:00.000000', '2015-12-31T00:00:00.000000',
                          '2016-01-31T00:00:00.000000', '2016-02-29T00:00:00.000000',
                          '2016-03-31T00:00:00.000000', '2016-04-30T00:00:00.000000',
                          '2016-05-31T00:00:00.000000', '2016-06-30T00:00:00.000000',
                          '2016-07-31T00:00:00.000000', '2016-08-31T00:00:00.000000',
                          '2016-09-30T00:00:00.000000', '2016-10-31T00:00:00.000000',
                          '2016-11-30T00:00:00.000000', '2016-12-31T00:00:00.000000',
                          '2017-01-31T00:00:00.000000', '2017-02-28T00:00:00.000000',
                          '2017-03-31T00:00:00.000000', '2017-04-30T00:00:00.000000',
                          '2017-05-31T00:00:00.000000', '2017-06-30T00:0

In [3]:
ex

'Between **2012-09-30** and **2014-06-30**, AAPL experienced a maximum drawdown of **-39.19%**. The drawdown lasted **1 years 9 months** until recovery.'

In [1]:
from helpers import indicators
indicators.compute_sharpe_sortino_calmar(db_path="data/market/market.duckdb", symbol="AAPL")

/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:494: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_close = df["close"].resample("M").last().dropna()


({'cagr_pct': 26.0,
  'vol_ann_pct': np.float64(26.53),
  'downside_dev_ann_pct': np.float64(14.66),
  'max_dd_pct': 39.19,
  'sharpe': 0.9,
  'sortino': 1.64,
  'calmar': 0.66},
 'Computed from monthly closes (price-only) from 2010-01-31 to 2025-07-31. Sharpe = (CAGR − rf) / σ; Sortino = (CAGR − rf) / σ_down; Calmar = CAGR / |MaxDD|. rf assumed 2.0%/yr. CAGR 26.0%, Vol 26.53%, Downside Dev 14.66%, Max DD 39.19%.')

In [24]:
from helpers import indicators
fig, note = indicators.plot_return_distribution(db_path="data/market/market.duckdb", symbol="NFLX", bins=90)

/Users/oluwadolaposalako/CryptoAdvisor/helpers/indicators.py:585: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [28]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'histnorm': 'probability density',
              'name': 'Monthly Returns',
              'nbinsx': 90,
              'opacity': 0.9,
              'type': 'histogram',
              'x': {'bdata': ('1JMMTzxqGECWRp2TmEcnQGT3KsRUEE' ... 'v8ykawGkAy2KWAB9olQOdEIc3B1yrA'),
                    'dtype': 'f8'}},
             {'mode': 'lines',
              'name': 'Normal PDF (mean/std-matched)',
              'type': 'scatter',
              'x': {'bdata': ('rWjqtigWTcB9oh7/UupMwE3cUkd9vk' ... 'uoInJTQBzb8YMNiFNAM77XX/idU0A='),
                    'dtype': 'f8'},
              'y': {'bdata': ('7NPo6QQc4j46zL0SNMfjPtd1CA4Nl+' ... 't7AZKTPlFk/xx5m5E+ilR/Rryqjz4='),
                    'dtype': 'f8'}}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': '0%',
                                'x': 0,
                                'xanchor': 'left',
                                'xref': 'x',
                                'y': 1,
                                'yanchor': 'top',
                                'yref': 'y domain'},
                               {'showarrow': False,
                                'text': 'Mean 3.82%',
                                'x': 3.819339684066256,
                                'xanchor': 'left',
                                'xref': 'x',
                                'y': 1,
                                'yanchor': 'top',
                                'yref': 'y domain'},
                               {'showarrow': False,
                                'text': '5th–95th pct',
                                'x': np.float64(-18.7519109337752),
                                'xanchor': 'left',
                                'xref': 'x',
                                'y': 1,
                                'yanchor': 'top',
                                'yref': 'y domain'}],
               'bargap': 0.02,
               'hovermode': 'x',
               'shapes': [{'line': {'dash': 'dash', 'width': 1},
                           'type': 'line',
                           'x0': 0,
                           'x1': 0,
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'},
                          {'line': {'dash': 'dot', 'width': 1},
                           'type': 'line',
                           'x0': 3.819339684066256,
                           'x1': 3.819339684066256,
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'},
                          {'fillcolor': 'LightGray',
                           'line': {'width': 0},
                           'opacity': 0.2,
                           'type': 'rect',
                           'x0': np.float64(-18.7519109337752),
                           'x1': np.float64(28.167373561276314),
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'}],
               'template': '...',
               'title': {'text': 'NFLX — Monthly Return Distribution'},
               'xaxis': {'title': {'text': 'Monthly return (%)'}},
               'yaxis': {'title': {'text': 'Density'}}}
})

In [27]:
print(note)

Computed from monthly closes (price-only) from 2010-01-31 to 2025-07-31. Skew = 0.73 (right/upside), Excess kurtosis = 5.45 (fat tails). Mean monthly return 3.82%, stdev 15.50%. 37.1% of months were negative. 5th–95th percentile range: -18.75% to 28.17%.


In [ ]:
from helpers import indicators
indicators.get_best_worst_month_and_hit_rate

In [1]:
from helpers import indicators
db = indicators.load_db(db_path="data/market/market.duckdb")
indicators.compute_momentum_12_1_with_fig(con=db, symbol="AAPL")

IOException: IO Error: Could not set lock on file "/Users/oluwadolaposalako/CryptoAdvisor/data/market/market.duckdb": Conflicting lock is held in /opt/anaconda3/envs/cryptoadvisor/bin/python3.10 (PID 21687) by user oluwadolaposalako. See also https://duckdb.org/docs/stable/connect/concurrency